In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import from_csv
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel

In [2]:
# Spark session & context
def init():
    spark = (SparkSession
         .builder
         .master('local')
         .appName('AndMalwareMLTest')
         # Add kafka package  
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2")\
         .getOrCreate())
    return spark

In [3]:
sc = init()
# Create stream dataframe setting kafka server, topic and offset option
def getReadStream(spark):
    df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "kafka-server:9092") \
      .option("startingOffsets", "earliest") \
      .option("subscribe", "MalModel") \
      .load()
    
    #df.selectExpr("CAST(value AS STRING)", "timestamp")
    dff = (df
    .withColumn("key", df["key"].cast(StringType()))
    .withColumn("value", df["value"].cast(StringType())))
    return dff


:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-688989a3-fde0-452a-86b0-6787c45fa2ef;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 379ms :: artifacts dl 11ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [4]:
from pyspark.ml.classification import DecisionTreeClassificationModel

Model = DecisionTreeClassificationModel.load("hdfs://namenode:9000/user/jovyan/model")
print(Model)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_583cb752ea02, depth=5, numNodes=27, numClasses=2, numFeatures=3


In [7]:
def foreach_batch_function(df, epoch_id):

    labelIndexer = StringIndexer(inputCol="Label", outputCol="indexedLabel").fit(df)
    print(labelIndexer)
    col = df.columns
    del col[-1]
    vecta = VectorAssembler(inputCols=col,outputCol="features")
    print(vecta)
    dataTemp = vecta.transform(df)
    #dataTemp = dataTemp.dropna()
    featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(dataTemp)
    dataTemp2 = featureIndexer.transform(dataTemp)
    #print(featureIndexer)
    predictions = Model.transform(dataTemp2)
    predictions.show()
    pass

In [9]:
df1 = getReadStream(sc)

df2 = df1.selectExpr("split(value,',')[0] as Flow_Duration" \
                 ,"split(value,',')[1] as Flow_IAT_Mean" \
                 ,"split(value,',')[2] as FP" \
                ,"split(value,',')[3] as Label" \
                    )



df3 = df2.withColumn("Flow_Duration", df2["Flow_Duration"].cast(IntegerType()))\
        .withColumn("Flow_IAT_Mean", df2["Flow_IAT_Mean"].cast(DoubleType()))\
        .withColumn("FP", df2["FP"].cast(IntegerType()))\
        .withColumn("Label", df2["Label"].cast(StringType()))\



query = (df3.writeStream\
        .foreachBatch(foreach_batch_function)\
        .outputMode('update')\
        .trigger(processingTime='3 seconds')\
        .start())

query.awaitTermination()


22/03/05 07:14:50 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-18293005-d891-45dc-a947-b434563637bf. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/03/05 07:14:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


StringIndexerModel: uid=StringIndexer_a3f339cf8b03, handleInvalid=error
VectorAssembler_ef871c133a43


22/03/05 07:14:52 ERROR Executor: Exception in task 0.0 in stage 11.0 (TID 11)
org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3918/0x000000084184f840: (struct<Flow_Duration_double_VectorAssembler_ef871c133a43:double,Flow_IAT_Mean:double,FP_double_VectorAssembler_ef871c133a43:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.serializefromobject_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterat

StreamingQueryException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.2-src.zip/py4j/clientserver.py", line 581, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 196, in call
    raise e
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 193, in call
    self.func(DataFrame(jdf, self.sql_ctx), batch_id)
  File "/tmp/ipykernel_51/2522311190.py", line 11, in foreach_batch_function
    featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(dataTemp)
  File "/usr/local/spark/python/pyspark/ml/base.py", line 161, in fit
    return self._fit(dataset)
  File "/usr/local/spark/python/pyspark/ml/wrapper.py", line 335, in _fit
    java_model = self._fit_java(dataset)
  File "/usr/local/spark/python/pyspark/ml/wrapper.py", line 332, in _fit_java
    return 